# Treating database

### Importing database

In [64]:
import json
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

from ast import literal_eval
from unidecode import unidecode

In [65]:
db_file = open('resume_base.json', 'rb')
db = json.loads(db_file.read())
db[0:2]

[{'id': 1,
  'title': 'Candidato de Contabilidade',
  'age': 49,
  'location': 'Centro, Curitiba - PR',
  'professional_goals_salary': '{"max": 0.0, "min": 0.0}',
  'professional_goals_contract_type': None,
  'professional_goals_working_hours': 'Período Integral',
  'education': '[{"dateEnd": "2003-11-01", "dateInit": "2003-02-01", "courseName": "Curso extra-curricular / Profissionalizante  em Contabilidade para executivos", "educationInstitutionName": "UNICENP em PR"}, {"dateEnd": "1989-12-01", "dateInit": "1985-02-01", "courseName": "Ensino Superior em Ciências contábeis", "educationInstitutionName": "FESP-Fundação de estudos Sociais do Parana em PR"}, {"dateEnd": "1984-11-01", "dateInit": "1982-02-01", "courseName": "Curso Técnico em Contabilidade", "educationInstitutionName": "CEPC-Centro de Estudos Profission.de Curitiba em PR"}]',
  'work_experience': '[{"field": "Contábil, Finanças, Economia", "level": null, "title": "CONTADOR", "dateEnd": "2006-12-01", "dateInit": "2005-03-01",

In [66]:
original_raw_data = pd.DataFrame(db)
raw_data = original_raw_data.copy()
raw_data = raw_data.set_index('id')
raw_data.head()

,age,complementary_data,education,language,location,marital_status,professional_goals_contract_type,professional_goals_salary,professional_goals_working_hours,sex,title,work_experience
id,,,,,,,,,,,,
1,49,"{""travel"": true, ""driving"": ""B"", ""vehicle"": ""C...","[{""dateEnd"": ""2003-11-01"", ""dateInit"": ""2003-0...","[{""level"": ""Intermediário"", ""language"": ""Inglê...","Centro, Curitiba - PR",None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Contabilidade,"[{""field"": ""Contábil, Finanças, Economia"", ""le..."
2,38,"{""travel"": true, ""driving"": ""A"", ""vehicle"": ""C...","[{""dateEnd"": ""2005-12-01"", ""dateInit"": ""2001-0...","[{""level"": ""Avançado"", ""language"": ""Inglês""}, ...",Santa Helena - PR,None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Zootecnia,"[{""field"": ""Agricultura, Pecuária, Veterinária..."
3,54,"{""travel"": true, ""driving"": ""B"", ""vehicle"": nu...","[{""dateEnd"": ""1999-10-01"", ""dateInit"": ""1999-0...","[{""level"": ""Intermediário"", ""language"": ""Inglê...","Centro, Curitiba - PR",None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Hotelaria,"[{""field"": ""Hotelaria, Turismo"", ""level"": null..."
4,41,"{""travel"": false, ""driving"": ""B"", ""vehicle"": ""...","[{""dateEnd"": ""2001-09-01"", ""dateInit"": ""1997-0...","[{""level"": ""Intermediário"", ""language"": ""Inglê...","Centro, Curitiba - PR",None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Marketing,"[{""field"": ""Marketing"", ""level"": null, ""title""..."
5,31,"{""travel"": false, ""driving"": null, ""vehicle"": ...","[{""dateEnd"": ""2004-05-01"", ""dateInit"": ""2004-0...","[{""level"": ""Intermediário"", ""language"": ""Portu...","Jardim Mediterrâneo, Colombo - PR",None,None,"{""max"": 0.0, ""min"": 0.0}",Período Integral,Não Especificado,Candidato de Instrumentação Industrial,"[{""field"": ""Industrial, Produção, Fábrica"", ""l..."


## Treating Data

In [67]:
treated_dataset = pd.DataFrame()
no_treatment_needed_attributes = ['age']
treated_dataset[no_treatment_needed_attributes] = raw_data[no_treatment_needed_attributes]

In [68]:
def swap_none_for_constant(data, column, fill_value, missing_values=None):
    imputer = SimpleImputer(missing_values=missing_values, strategy='constant', fill_value=fill_value)
    imputer.fit(data[column].values.reshape(-1,1))
    data[column] = imputer.transform(data[column].values.reshape(-1,1))

In [69]:
def one_edit_apart(s1, s2):
    if abs(len(s1) - len(s2)) > 1:
        return False
    if len(s1) > len(s2):
        bigger = s1
        smaller = s2
    else:
        bigger = s2
        smaller = s1

    equal_length_strings = len(s1) == len(s2)
    saw_difference = False
    i = 0
    j = 0
    while i < len(smaller): 
        if (bigger[j] != smaller[i]):
            if (saw_difference):
                return False
            saw_difference = True
            if (not equal_length_strings):
                i = i - 1
        i = i + 1
        j = j + 1
    return True

### Professional Goals Salary Treat

For a first approach as it is a litterally numerical attribute we will just divide into a max and min attribute

In [70]:
raw_data['professional_goals_salary'].unique()

array(['{"max": 0.0, "min": 0.0}', None, '{"max": 3000.0, "min": 1200.0}',
       ..., '{"max": 2450.0, "min": 2300.0}',
       '{"max": 14000.0, "min": 7000.0}',
       '{"max": 4800.0, "min": 2800.0}'], dtype=object)

Tem valores de None, entao a primeira abordagem trivial é só substituir por {"max": 0.0, "min":0.0}, pode ser interessante usar a média dos valores futuramente (inclusive substituindo as atuais ocorrencias de {"max": 0.0, "min":0.0})

In [71]:
display(raw_data['professional_goals_salary'].isnull().values.any())
swap_none_for_constant(raw_data, 'professional_goals_salary', {"max": 0.0, "min":0.0})
raw_data['professional_goals_salary'].isnull().values.any()

True

False

In [72]:
treated_dataset['professional_goals_salary_min'] = raw_data['professional_goals_salary'].astype(str).map(lambda v: literal_eval(v)['min'])
treated_dataset['professional_goals_salary_max'] = raw_data['professional_goals_salary'].astype(str).map(lambda v: literal_eval(v)['max'])

In [73]:
treated_dataset[['professional_goals_salary_min', 'professional_goals_salary_max']].sample(5)

,professional_goals_salary_min,professional_goals_salary_max
id,,
65826,2000.0,3500.0
69960,900.0,1500.0
11566,800.0,1200.0
13706,1200.0,2000.0
93622,1500.0,3000.0


### Professional Goals Contract Type Treat

In [74]:
raw_data['professional_goals_contract_type'].astype(str).apply(unidecode).unique()

array(['None', 'Estagio', 'Efetivo - CLT', 'Outros', 'Temporario',
       'Autonomo', 'Trainee', 'Cooperado', 'Prestador de Servicos (PJ)',
       'Estgio', 'Efetivo  CLT'], dtype=object)

Necessario unificar termos com distancia de edição até 1 e incluir não informado nas situações com None

In [75]:
swap_none_for_constant(raw_data, 'professional_goals_contract_type', "Nao Informado")

In [76]:
raw_data['professional_goals_contract_type'].astype(str).apply(unidecode).unique()

array(['Nao Informado', 'Estagio', 'Efetivo - CLT', 'Outros',
       'Temporario', 'Autonomo', 'Trainee', 'Cooperado',
       'Prestador de Servicos (PJ)', 'Estgio', 'Efetivo  CLT'],
      dtype=object)

In [77]:
core_contracts_types = ['Nao Informado', 'Estagio', 'Efetivo - CLT', 'Outros',
       'Temporario', 'Autonomo', 'Trainee', 'Cooperado', 'Prestador de Servicos (PJ)']

treated_dataset['professional_goals_contract_type'] = raw_data['professional_goals_contract_type'].astype(str).apply(unidecode)
clean_contract_type = np.array(treated_dataset['professional_goals_contract_type'])

for contract_type in core_contracts_types:
    for i in range(0,len(clean_contract_type)):
        if one_edit_apart(clean_contract_type[i], contract_type):
            clean_contract_type[i] = contract_type

treated_dataset['professional_goals_contract_type'] = clean_contract_type

In [78]:
treated_dataset['professional_goals_contract_type'].unique()

array(['Nao Informado', 'Estagio', 'Efetivo - CLT', 'Outros',
       'Temporario', 'Autonomo', 'Trainee', 'Cooperado',
       'Prestador de Servicos (PJ)'], dtype=object)

### Professional Goals Working Hours

In [79]:
raw_data['professional_goals_working_hours'].astype(str).apply(unidecode).unique()

array(['Periodo Integral', 'Parcial manhas', 'None', 'Parcial tardes',
       'Parcial noites', 'Noturno', 'Perodo Integral', 'Parcial manhs'],
      dtype=object)

In [80]:
swap_none_for_constant(raw_data, 'professional_goals_working_hours', "Nao Informado")

In [81]:
core_working_hours = ['Periodo Integral', 'Parcial manhas', 'None', 'Parcial tardes',
                       'Parcial noites', 'Noturno']

In [82]:
treated_dataset['professional_goals_working_hours'] = raw_data['professional_goals_working_hours'].astype(str).apply(unidecode)
clean_working_hours = np.array(treated_dataset['professional_goals_working_hours'])

for working_hours in core_working_hours:
    for i in range(0,len(clean_working_hours)):
        if one_edit_apart(clean_working_hours[i], working_hours):
            clean_working_hours[i] = working_hours

treated_dataset['professional_goals_working_hours'] = clean_working_hours

In [83]:
treated_dataset['professional_goals_working_hours'].unique()

array(['Periodo Integral', 'Parcial manhas', 'Nao Informado',
       'Parcial tardes', 'Parcial noites', 'Noturno'], dtype=object)

### Complementary Data Treat

In [84]:
complementary_data_keys = list(json.loads(raw_data['complementary_data'].values[0]).keys())

for key in complementary_data_keys:
    treated_dataset['complementary_data_' + key] = raw_data['complementary_data'].map(lambda v: json.loads(v)[key])

Vehicle e Driving precisam de um tratamento extra para remover Nones e transformar strings que representam multiplos atributos em lista e futuramente em uma feature cada

In [85]:
treated_dataset['complementary_data_driving'].unique()

array(['B', 'A', None, 'A, B', 'E', 'C', 'D', 'B, A', 'A, B, C, D, E',
       'A, B, C, D', 'B, D', 'A, C', 'A, D', 'A, B, C', 'A, E', 'B, B',
       'B, C', 'A, A', 'C, A, B', 'B, A, C, D', 'B, C, D', 'B, D, A',
       'D, A', 'D, B', 'A, B, D', 'A, C, B', 'C, E', 'A, B, D, C',
       'B, A, D', 'D, B, C', 'D, E', 'A, D, B, C, E', 'D, A, B, C',
       'C, A', 'B, D, A, C', 'A, D, B, C', 'E, A', 'D, B, C, A',
       'A, B, D, E, C', 'A, B, A, B, A, B', 'A, C, D', 'E, A, B, C, D',
       'A, B, C, E'], dtype=object)

### Location Treat

In [86]:
experimentation = raw_data.copy()
experimentation['count'] = 1
experimentation.groupby('location').sum().sort_values('count', ascending=False)

,age,count
location,,
"Centro, Curitiba - PR",334116,13703
"Cidade Industrial, Curitiba - PR",227783,9729
"Sítio Cercado, Curitiba - PR",151700,6510
"Cajuru, Curitiba - PR",117833,4943
"Boqueirão, Curitiba - PR",99511,4121
"Uberaba, Curitiba - PR",85292,3609
"Xaxim, Curitiba - PR",76205,3174
"Pinheirinho, Curitiba - PR",70399,2956
"Tatuquara, Curitiba - PR",61337,2718


In [87]:
test_db = [ {'age':resume['age'], 'id': resume['id'], 'professional_goals_working_hours': resume['professional_goals_working_hours'], 'sex':resume['sex'], 'title': resume['title']} for resume in db]

In [88]:
test_db[0]

{'age': 49,
 'id': 1,
 'professional_goals_working_hours': 'Período Integral',
 'sex': 'Não Especificado',
 'title': 'Candidato de Contabilidade'}

### Output Treated Dataset

In [89]:
#with open('treated_data.json', 'w') as outfile:
#    json.dump(test_db, outfile)

In [90]:
output_dataset = treated_dataset.reset_index()
output_dataset.to_csv('treated_dataset.csv')